#End-to-End Loan Eligibility Classification

The goal of this project will be to determine loan eligiblity based on a number of features. The intention is to go through all necessary steps of a machine learning classification problem from start to finish.

# Data Cleanup

We have gotten some insights on how to move forward from our exploratory data analysis, now let's make a pipeine to automatically transform each columns. We can manally remove the outliers mentioned previously as there are only a few:

In [ ]:
loan_data = loan_data[loan_data['person_age']<100]
loan_data = loan_data[loan_data['person_emp_exp']<=50]

In [ ]:
loan_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 35992 entries, 25180 to 15795
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   person_age                      35992 non-null  float64
 1   person_gender                   35992 non-null  object 
 2   person_education                35992 non-null  object 
 3   person_income                   35992 non-null  float64
 4   person_emp_exp                  35992 non-null  int64  
 5   person_home_ownership           35992 non-null  object 
 6   loan_amnt                       35992 non-null  float64
 7   loan_intent                     35992 non-null  object 
 8   loan_int_rate                   35992 non-null  float64
 9   loan_percent_income             35992 non-null  float64
 10  cb_person_cred_hist_length      35992 non-null  float64
 11  credit_score                    35992 non-null  int64  
 12  previous_loan_defaults_on_file  3

In [ ]:
loan_data = train_set.drop('loan_status', axis=1)
loan_labels = train_set['loan_status'].copy()

In [ ]:
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer, OrdinalEncoder, LabelBinarizer, MinMaxScaler
from sklearn.compose import ColumnTransformer, make_column_selector

def log_transform(x):
    return np.log(x + 1e-10)

log_transformer = make_pipeline(
    SimpleImputer(strategy='most_frequent'),
    FunctionTransformer(log_transform, feature_names_out='one-to-one'),
    StandardScaler()
)

category_transformer = make_pipeline(
    SimpleImputer(strategy='most_frequent'),
    OneHotEncoder(handle_unknown='ignore')
)

numeric_transformer = make_pipeline(
    SimpleImputer(strategy='most_frequent'),
    StandardScaler()
)

education_order = [['High School', 'Associate', 'Bachelor', 'Master',
                    'Doctorate']]

education_transformer = make_pipeline(
    SimpleImputer(strategy='most_frequent'),
    OrdinalEncoder(categories=education_order)
)

binary_transformer = make_pipeline(
    SimpleImputer(strategy='most_frequent'),
    OneHotEncoder(handle_unknown='ignore', drop='if_binary'),
)

preprocessing = ColumnTransformer([
    ('log', log_transformer, ['person_age', 'person_income', 'person_emp_exp',
                              'loan_amnt','loan_int_rate',
                              'loan_percent_income', 'loan_int_rate',
                              'cb_person_cred_hist_length']),
    ('cat', category_transformer, ['person_home_ownership','loan_intent']),
    ('edu', education_transformer, ['person_education']),
    ('bin', binary_transformer, ['person_gender',
                                 'previous_loan_defaults_on_file'])],
    remainder=numeric_transformer, force_int_remainder_cols=False)